In [1]:
%load_ext autoreload  
%autoreload 2 
%matplotlib inline

In [16]:
import octopus
from src import *

In [3]:
print(octopus.__file__)

/Users/garylai/Desktop/kaggle/petfinder/octopus/__init__.py


In [6]:
import numpy as np
import pandas as pd
import timm
from sklearn import metrics
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations

In [7]:
class Config: 
    debug = True
    epochs = 5
    batch_size = 4
    image_size = 256

In [8]:
data_path = "./data"
df_train = pd.read_csv(f'{data_path}/train.csv')
if Config.debug: 
    df_train = df_train[:8]
train_img_paths = [f"{data_path}/train/{id}.jpg" for id in df_train["Id"].values]

In [9]:
train_aug = train_aug = albumentations.Compose(
    [
        albumentations.Resize(Config.image_size, Config.image_size, p=1),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

dense_features = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
    'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
]

train_ds = PawpularDataset(
    image_paths=train_img_paths,
    dense_features=df_train[dense_features].values,
    targets=df_train.Pawpularity.values,
    augmentation=train_aug
)

In [10]:
train_ds[0]['image'].shape

(256, 256, 3)

In [11]:
train_dl = DataLoader(train_ds, batch_size=Config.batch_size, shuffle=False)

In [12]:
# examine the batches
for x in train_dl:
    print({
        "image": x['image'].shape,
        "dense_features": x['dense_features'].shape,
        "target": x["target"].shape
    })

{'image': torch.Size([4, 256, 256, 3]), 'dense_features': torch.Size([4, 12]), 'target': torch.Size([4])}
{'image': torch.Size([4, 256, 256, 3]), 'dense_features': torch.Size([4, 12]), 'target': torch.Size([4])}


In [ ]:
# classifier = timm.create_model("tf_efficientnet_b0_ns", pretrained=True, in_chans=3).classifier; print(classifier)

In [17]:
model = PawpularModel()

NameError: name 'ProgressBar' is not defined

In [ ]:
cbs = [octopus.ProgressBar(), octopus.TrackLoss(save_model=False, path="", model_name=f"pawpular"), octopus.LrRecorder(len(param_groups))]

In [ ]:
model.fit(
    train_ds,
    valid_dataset=train_ds,
    train_bs=Config.batch_size,
    valid_bs=2*Config.batch_size,
    device="cpu",
    epochs=Config.epochs,
    # callbacks=[es],
    # fp16=True,
)